In [0]:
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import min, max, avg, col, udf
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.appName("office_data_project").getOrCreate()

In [0]:
office_data = spark.read.options(header=True, inferSchema=True).csv('/FileStore/tables/OfficeDataProject.csv')
office_data.persist(StorageLevel.MEMORY_AND_DISK_DESER)
office_data.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
##### total number of employees #####
office_data.count()

Out[158]: 1000

In [0]:
##### total number of departments #####
office_data.dropDuplicates(['department']).count()

Out[159]: 6

In [0]:
##### department names #####
office_data.select('department').dropDuplicates(['department']).show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [0]:
##### total number of employees in each department #####
office_data.groupBy(office_data.department).count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



In [0]:
##### total number of employees in each state #####
office_data.groupBy(office_data.state).count().show()

+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



In [0]:
##### total number of employees in each state in each department #####
office_data.groupBy(office_data.state, office_data.department).count().sort(office_data.state).show()

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   AK|Purchasing|   30|
|   AK|     Sales|   38|
|   AK|  Accounts|   37|
|   AK|        HR|   25|
|   AK|   Finance|   37|
|   AK| Marketing|   42|
|   CA|     Sales|   42|
|   CA| Marketing|   33|
|   CA|   Finance|   35|
|   CA|  Accounts|   35|
|   CA|Purchasing|   32|
|   CA|        HR|   28|
|   LA|        HR|   41|
|   LA|     Sales|   35|
|   LA|   Finance|   29|
|   LA|Purchasing|   45|
|   LA|  Accounts|   29|
|   LA| Marketing|   26|
|   NY|  Accounts|   34|
|   NY|     Sales|   27|
+-----+----------+-----+
only showing top 20 rows



In [0]:
##### min/max salaries in each department, sorted in ascending order #####
office_data.groupBy(office_data.department)\
    .agg(min('salary').alias('min_salary'), max('salary').alias('max_salary'))\
    .orderBy(col('max_salary'), col('min_salary') ).show()

+----------+----------+----------+
|department|min_salary|max_salary|
+----------+----------+----------+
|  Accounts|      1007|      9890|
|   Finance|      1006|      9899|
| Marketing|      1031|      9974|
|        HR|      1013|      9982|
|     Sales|      1103|      9982|
|Purchasing|      1105|      9985|
+----------+----------+----------+



In [0]:
##### name of NY employees in finance department whose bonuses > average NY bonuses #####
ny_office_data = office_data.filter(office_data.state == 'NY')
avg_bonus_ny = ny_office_data.groupBy('state')\
    .agg(avg('bonus').alias('avg_bonus') )\
    .select('avg_bonus').collect()[0]['avg_bonus']      # use wrapped RDD to return bonus as float val

ny_office_data.filter((ny_office_data.department == 'Finance' ) & (ny_office_data.bonus > avg_bonus_ny) )\
    .select('employee_name').show()

+--------------------+
|       employee_name|
+--------------------+
|       Vivan Sifford|
|      Herder Gallman|
|          Nena Rocha|
|       Leif Lemaster|
|Ellingsworth Meli...|
|        Escoto Gilma|
|     Georgeanna Laub|
|     Durio Tenenbaum|
|       Juliana Grigg|
|        Tiffani Benz|
|          Nitz Ilana|
|   Phylicia Antonina|
|         Durio Janey|
|       Melissia Jere|
|      Yukiko Kreamer|
|      Nena Kensinger|
|      Antonina Ilana|
+--------------------+



In [0]:
##### raise salary by $500 for all employees over 45 #####
def update_salary(salary, age):
    if age > 45:
        return salary + 500
    else:
        return salary 
update_salary_udf = udf(lambda salary, age: update_salary(salary, age), IntegerType())

office_data.withColumn('salary', update_salary_udf(office_data.salary, office_data.age)).show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  6217| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  4204| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
##### save df of employees over 45 to file #####

employees_over_45 = office_data.filter(office_data.age > 45)
employees_over_45.show()
employees_over_45.write.csv('/FileStore/tables/OfficeData/over_45_output/')

+-----------+------------------+----------+-----+------+---+-----+
|employee_id|     employee_name|department|state|salary|age|bonus|
+-----------+------------------+----------+-----+------+---+-----+
|       1003|       Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1008| Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1011|  Barringer Escoto|Purchasing|   WA|  1685| 49| 1706|
|       1018|Vankirk Jacquelyne|Purchasing|   NY|  8636| 47| 1192|
|       1025|   Dionne Lemaster|     Sales|   AK|  5134| 48| 1356|
|       1030|        Trena Benz|  Accounts|   NY|  4376| 49| 1624|
|       1039|      Dynes Katlyn|  Accounts|   AK|  3039| 48|  834|
|       1058|      Clune Norene|   Finance|   AK|  1605| 49|  801|
|       1074|      Rocha Dionne|  Accounts|   CA|  3470| 49|  706|
|       1088|       Imai Locust|     Sales|   NY|  9982| 49| 1809|
|       1090| Clemencia Rudolph|   Finance|   NY|  1296| 50| 1209|
|       1099|    Zollner Marvis|Purchasing|   CA|  4230| 50|  